In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml calibration_config

experiment:
  seed: 42
    
dataloader:
  batch_size: 1 
  num_workers: 1
  pin_memory: True 

optim: 
  _class: torch.optim.Adam
  lr: 3.0e-4
  weight_decay: 0.0 

train:
  epochs: 1000 
  eval_freq: 10 
  pretrained_dir: /storage/vbutoi/scratch/ESE/training/10_23_23_Dense_WMH 

log:
  checkpoint_freq: 5 
  root: '?'
  metrics:
    ece_loss:
      _fn: ese.experiment.metrics.ece_loss
      from_logits: True
    elm_loss:
      _fn: ese.experiment.metrics.elm_loss
      from_logits: True

loss_func: 
  _class: ionpy.loss.PixelCELoss 
  from_logits: False
  batch_reduction: 'mean' 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibrator_model_cfg  

model:
  _class: '?'
  num_classes: 2
  image_channels: 1

<IPython.core.display.Javascript object>

In [4]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.experiment.callbacks.ShowPredictions
  epoch:
    - ese.experiment.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: ece_loss 
        phase: val

<IPython.core.display.Javascript object>

In [5]:
# Setup direcrtories
calibration_root = '/storage/vbutoi/scratch/ESE/calibration'

# WMH CONFIG
exp_name = '01_02_24_WMH_lowlr'

# Setup the root.
exp_root = f'{calibration_root}/{exp_name}'

# Create the ablation options
option_set = [
    {
        'log.root': [exp_root],
        'optim.lr': [1e-4],
        'model._class': ['ese.experiment.models.calibrators.Temperature_Scaling',
                         'ese.experiment.models.calibrators.Vector_Scaling',
                         'ese.experiment.models.calibrators.Dirichlet_Scaling',
                         'ese.experiment.models.calibrators.LTS'],
    },
]

In [6]:
from ese.scripts.utils import get_option_product
from ionpy.util import Config

# Assemble base config
base_cfg = Config(calibration_config).update([calibrator_model_cfg, callbacks_cfg])

# Get the configs
cfgs = get_option_product(exp_name, option_set, base_cfg)

In [7]:
len(cfgs)

4

## Running Jobs

In [8]:
# from ese.experiment.experiment import run_ese_exp, PostHocExperiment

# run_ese_exp(
#     config=cfgs[0], 
#     experiment_class=PostHocExperiment,
#     gpu='3',
#     run_name='debug',
#     show_examples=False,
#     track_wandb=True
# )

In [9]:
# # FOR SUBMISSION
from ese.experiment.experiment import submit_ese_exps, PostHocExperiment

submit_ese_exps(
    exp_root=exp_root,
    experiment_class=PostHocExperiment,
    config_list=cfgs,
    available_gpus=['0', '1', '2', '3'],
    track_wandb=True
)

Initalized SliteRunner
Submitted job id: 2496686.
Submitted job id: 2496692.
Submitted job id: 2496731.
Submitted job id: 2496739.
